In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import path
import os

In [23]:
yugi_test = pd.read_csv('Yugioh_test_16.csv')

In [3]:
yugi_test.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Name', 'Card type', 'Attribute',
       'Monster Type', 'Level/Rank', 'ATK/DEF', 'Passcode',
       'Materials / Ritual spell', 'Effect type', 'Effect', 'Spell/Trap type',
       'TCG sets', 'Ban_list', 'Number of sets', 'Link Arrows',
       'Pendulum Scale', 'Rarities', 'Set Name', 'Release Date',
       'Yugioh Prices Link'],
      dtype='object')

In [4]:
def yugioh_card_prices_link(card_code, rarity):
    if ' ' in rarity:
        hello = rarity.replace(' ','%20')
        a = 'https://yugiohprices.com/price_history/' + card_code + '?rarity=' + hello
        return a
    else:
        a = 'https://yugiohprices.com/price_history/' + card_code + '?rarity=' + rarity
        return a

In [6]:
links = []
for i in range(0, len(yugi_test['Name'])):
    b = yugioh_card_prices_link(yugi_test['TCG sets'][i], yugi_test['Rarities'][i])
    links.append(b)
    
yugi_test['Yugioh Prices Link'] = links

In [5]:
links = list(yugi_test['Yugioh Prices Link'])

In [27]:
def card_price(url):
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    Misa = soup.find_all("table", attrs = {'class': 'sortable'})
    Jeff = soup.find_all("h1")
    Jeff_2 = soup.find_all("b")
    Jeff_3 = Jeff + Jeff_2
    if ('NoMethodError' in Jeff_3[0].text) or ('Error' in Jeff_3[1].text) or (Jeff_3 == []):
        return('N/A')
    Misa = soup.find_all("table", attrs = {'class': 'sortable'})
    #Parsing table entries
    b = Misa[0].find_all("td")
    #Getting all the entries in the price table
    there = []
    for hello in b:
        there.append(hello.text)
    #Cleaning things up
    there_2 = []
    for hello in there:
        if '\n' in hello:
            hello_2 = hello.replace('\n','')
            there_2.append(hello_2)
        else:
            there_2.append(hello)
    Date = []
    lowest_price = []
    highest_price = []
    average_price = []
    price_shift = []

    for i in range(0, len(there_2)):
        if i % 5 == 0:
            Date.append(there_2[i])
        elif i % 5 == 1:
            lowest_price.append(there_2[i])
        elif i % 5 == 2:
            highest_price.append(there_2[i])
        elif i % 5 == 3:
            average_price.append(there_2[i])
        else:
            price_shift.append(there_2[i])
    df = pd.DataFrame({'Date': Date, 'Lowest Price': lowest_price, 'Highest Price': highest_price, 
                   'Average Price': average_price, 'Price Shift': price_shift})
    return df

In [44]:
Wilbur = []
bruh = []
for link in list(yugi_test['Yugioh Prices Link']):
    print(link)
    Lilo = card_price(link)
    Wilbur.append(Lilo)
    bruh.append('ok')
    print(len(bruh))

https://yugiohprices.com/price_history/CRMS-EN014?rarity=Common
1
https://yugiohprices.com/price_history/CRMS-EN013?rarity=Common
2


In [45]:
card_to_prices = {yugi_test['Name'][i] + yugi_test['TCG sets'][i] + yugi_test['Rarities'][i] : Wilbur[i] for i in range(0, len(Wilbur))}

In [48]:
for i in range(0, len(Wilbur)):
    a = list(card_to_prices.keys())[i]
    if (type(card_to_prices[a]) == pd.core.frame.DataFrame):
        if not os.path.exists('Yugioh_Card_Price_Data'):
                os.makedirs('Yugioh_Card_Price_Data')
        if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com'):
            os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com')
        if '\"' in yugi_test['Name'][i]:
            jeff = yugi_test['Name'][i].replace('\"', '\'')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        elif ('?' in yugi_test['Name'][i]):
            jeff = yugi_test['Name'][i].replace('?', '')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        elif ('!' in yugi_test['Name'][i]):
            jeff = yugi_test['Name'][i].replace('!', '')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        elif (':' in yugi_test['Name'][i]):
            jeff = yugi_test['Name'][i].replace(':', '')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        elif ('★' in yugi_test['Name'][i]):
            jeff = yugi_test['Name'][i].replace('★', '')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        elif ('Ω' in yugi_test['Name'][i]):
            jeff = yugi_test['Name'][i].replace('Ω', 'Omega')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        elif ('α' in yugi_test['Name'][i]):
            jeff = yugi_test['Name'][i].replace('α', 'Alpha')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        elif ('☆' in yugi_test['Name'][i]):
            jeff = yugi_test['Name'][i].replace('☆', '')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        elif ('・' in yugi_test['Name'][i]):
            jeff = yugi_test['Name'][i].replace('・', '')
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff)):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(jeff))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(jeff, 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
        
        
        else:
            if not os.path.exists('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(yugi_test['Name'][i])):
                os.makedirs('Yugioh_Card_Price_Data/From_yugiohprices.com/{}'.format(yugi_test['Name'][i]))
            card_to_prices[a].to_csv('Yugioh_Card_Price_Data/From_yugiohprices.com/{}/{}_{}.csv'.format(yugi_test['Name'][i], 
                                                                        yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))
    else:
        print('No info for {} {} {}'.format(yugi_test['Name'][i], yugi_test['TCG sets'][i], yugi_test['Rarities'][i]))

In [49]:
yugi_test = pd.read_csv('Yugioh_test_16.csv')
yugi_test = yugi_test[yugi_test['Name'].str.contains('・')]
yugi_test = yugi_test.reset_index()
links = list(yugi_test['Yugioh Prices Link'])

In [43]:
print(yugi_test)

   index  Unnamed: 0  Unnamed: 0.1               Name Card type Attribute  \
0  18079       18079         18197   Spell Reactor・RE  Monster      DARK    
1  20296       20296         20423  Trap Reactor・Y FI  Monster      DARK    

       Monster Type  Level/Rank     ATK/DEF  Passcode  ... Spell/Trap type  \
0  Machine / Effect         3.0  1200 / 900  15175429  ...             NaN   
1  Machine / Effect         4.0  800 / 1800  52286175  ...             NaN   

     TCG sets    Ban_list Number of sets Link Arrows Pendulum Scale  Rarities  \
0  CRMS-EN014  Unlimited               1         NaN            NaN    Common   
1  CRMS-EN013  Unlimited               1         NaN            NaN    Common   

                                       Set Name   Release Date  \
0  Crimson Crisis Sneak Peek Participation Card  March 3, 2009   
1  Crimson Crisis Sneak Peek Participation Card  March 3, 2009   

                                  Yugioh Prices Link  
0  https://yugiohprices.com/price_h